In [65]:
import numpy as np
from pandas import Series, DataFrame
import pandas as pd
from pandas import read_html

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import time

In [3]:
#Constants for accessing DataFrame data pulled from web
QB = 0
RB = 1
WR = 2
TE = 3
DST = 4
K = 5

DATA = 0
KEYS = 1

In [4]:
#Set to create unique in house DataFrame 

tupOffense = [('', 'Game'),        
              ('', 'Pts'),
              ('Passing', 'Att'),
              ('Passing', 'Cmp'),
              ('Passing', 'Yds'),
              ('Passing', 'TD'),
              ('Passing', 'Int'),
              ('Passing', '2Pt'),
              ('Rushing', 'Att'),
              ('Rushing', 'Yds'),
              ('Rushing', 'TD'),
              ('Rushing', '2Pt'),
              ('Receiving', 'Rec'),
              ('Receiving', 'Yds'),
              ('Receiving', 'TD'),
              ('Receiving', '2Pt'),
              ('Fumbles', 'FL'),
              ('Fumbles', 'TD')
             ]   

tupDef = [('', 'Opp'),
          ('', 'Pts*'),
          ('Def', 'Sack'), 
          ('Def', 'Int'),
          ('Def', 'Saf'), 
          ('Def', 'FR'), 
          ('Def', 'Blk'), 
          ('Def', 'TD'), 
          ('Def', 'PA'),
          ('Def', 'PassYds/G'), 
          ('Def', 'RushYds/G'), 
          ('Def', 'TotYds/G')
         ]

tupKicker = [('', 'Game'), 
             ('', 'Pts*'),          
             ('Kicking', 'XPA'),
             ('Kicking', 'XPM'),
             ('Kicking', 'FGA'),
             ('Kicking', 'FGM'),
             ('Kicking', '50+'),
            ]

#Set to create unique in house DataFrame of master Dataframe that will have all players and seasons
masterTup = [('', 'PPR'),
             ('', 'Pos'),
             ('', 'Team'),
             ('', 'Bye'),
             ('', 'Pts'),
             ('', 'Week'),
             ('', 'Year'),
             ('', 'Game'),
             ('', 'Opp'),
             ('Passing', 'Att'),
             ('Passing', 'Cmp'),
             ('Passing', 'Yds'),
             ('Passing', 'TD'),
             ('Passing', 'Int'),
             ('Passing', '2Pt'),
             ('Rushing', 'Att'),
             ('Rushing', 'Yds'),
             ('Rushing', 'TD'),
             ('Rushing', '2Pt'),
             ('Receiving', 'Rec'),
             ('Receiving', 'Yds'),
             ('Receiving', 'TD'),
             ('Receiving', '2Pt'),
             ('Fumbles', 'FL'),
             ('Fumbles', 'TD'),
             ('Defense', 'Sack'),
             ('Defense', 'Int'),
             ('Defense', 'Saf'),
             ('Defense', 'FR'), 
             ('Defense', 'Blk'),
             ('Defense', 'TD'),
             ('Defense', 'PA'),
             ('Defense', 'PassYds/G'),
             ('Defense', 'RushYds/G'),
             ('Defense', 'TotYds/G'),
             ('Kicking', 'XPA'),
             ('Kicking', 'XPM'),
             ('Kicking', 'FGA'),
             ('Kicking', 'FGM'),
             ('Kicking', '50+'),
            ]

In [5]:
#Set to change the index names 

indexDEF = ['Bye', 'Pts*', 'Sack', 'Int', 'Saf', 'FR', 'Blk', 'TD', 'PA',
       'PassYds/G', 'RushYds/G', 'TotYds/G']

indexKicker = ['Bye', 'Pts*', 'XPA', 'XPM', 'FGA', 'FGM', '50+']


In [6]:
#Load data from web for 2017 season
def loadWebData(frameArray):
    url_qb_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=QB&yr=2017&wk=all&rules=1'
    url_rb_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=RB&yr=2017&wk=all&rules=1'
    url_wr_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=WR&yr=2017&wk=all&rules=1'
    url_te_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=TE&yr=2017&wk=all&rules=1'
    url_def_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=DST&yr=2017&wk=all&rules=1'
    url_kicker_season17 = 'https://www.footballdb.com/fantasy-football/index.html?pos=K&yr=2017&wk=all&rules=1'
    
    #Array [0] is where the data is stored. [1] stores the rules 
    frameArray.insert(QB, pd.io.html.read_html(url_qb_season17)[0])
    frameArray.insert(RB, pd.io.html.read_html(url_rb_season17)[0])
    frameArray.insert(WR, pd.io.html.read_html(url_wr_season17)[0])
    frameArray.insert(TE, pd.io.html.read_html(url_te_season17)[0])
    frameArray.insert(DST, pd.io.html.read_html(url_def_season17)[0]) 
    frameArray.insert(K, pd.io.html.read_html(url_kicker_season17)[0]) 

In [7]:
#Fix name that contains the next line string '\xa0'
#nameIndex: Empty index that will contain the fixed name
#data: Pandas Series type that contains the names of each player
def nameFix(nameIndex, data):
    #Get name from Player column and save to nameIndex
    NAME_FIX_OFFSET = 3
    index = nameIndex

    for i in data:
        index.append(i)


    size = len(index)

    for i in range(size):
        a1 = index[i]
        count = 0

        for x in a1:
            count += 1
            if x == u'\xa0':
                break

        count -= NAME_FIX_OFFSET
        index[i] = a1[:count]
    
    return index

In [8]:
def renameIndex(dframe, teamName):

    d1 = dict()
    index = 0

    for i in teamName:
        d1[index] = i
        index += 1
        
    dframe = (dframe.rename(d1, axis = 'index'))
    return(dframe)
    

In [9]:
def trimName(teamName):    
    arryIndex = len(teamName)
    
    
    for a1 in range(arryIndex):
        #a1 is index for the string array teamName 
        strLen = len(teamName.iloc[a1])
        for b1 in teamName.iloc[a1][::-1]:
            if b1.isupper():
                strLen -= 1
            else:
                break
                
        teamName.iat[a1] = (teamName.iloc[a1][:strLen])
       
    return teamName

In [11]:
def createIndex(keyList):
    position = keyList['Pos'] 
    index = []
    
    if(position=='QB' or position=='RB' or position=='TE' or position=='WR' ):
        index = pd.MultiIndex.from_tuples(tupOffense)
    elif(position == 'K'):
        index = pd.MultiIndex.from_tuples(tupKicker)
    elif(position == 'DST'):
        index = pd.MultiIndex.from_tuples(tupDef)
        
    return index

In [208]:

#Pulls data from web as dataframes and stores in a frameArray and returns the array
def pullDataWeb():
    ffb_year = [i for i in range(2010, 2018)]
    ffb_stdscore = [1,0]
    ffb_week = [i for i in range(1,18)] 
    ffb_pos = ['QB', 'RB', 'WR', 'TE', 'K', 'DST']
    frameArray = []

    ffb_d1 = {'StdScore': ffb_stdscore, 'Year':ffb_year, 'Week': ffb_week, 'Pos': ffb_pos}

    keyList = []
    count = 0
    url = []


    #Create url string for every week. Save scoring, year, week, and pos for future reference
    for scoring in ffb_d1['StdScore']:
        for year in ffb_d1['Year']:
            for week in ffb_d1['Week']:
                for pos in ffb_d1['Pos']:
                    url.append("https://www.footballdb.com/fantasy-football/index.html?pos={p}&yr={y}&wk={w}&rules={r}"\
                    .format(p=pos, y=year, w=week, r=scoring))
                    keyList.append({'Stdscore':scoring, 'Year':year, 'Week':week, 'Pos':pos})
                    
    #frame array will be a list of tuples. first value of tuple is dframe. second is
    #dictionary that contains the keys
    for i in url:
        frameArray.append((pd.io.html.read_html(i)[0], keyList[count]))
        count += 1 
        
    return frameArray



In [181]:
def setDataFrame_old(dframe, keyList, index):
    nameIndex = []  #This is the name of the team or player
    dataMatrix = [] #This is the data going into dataframe
    position = keyList['Pos'] #Get the position of the player
    dkeylist = {
            ('','Pos'): keyList['Pos'], 
            ('', 'PPR'): ('No' if keyList['StdScore'] else 'Yes'),
            ('', 'Week'): keyList['Week'],
            ('', 'Year'): keyList['Year']
         }
    
    
    #Get the dimension of the frame. shape[0] will be the number of rows
    shape = dframe.shape
    
    #Go through each Series in the DataFrame from 0 to the number of rows
    #From each Series, extact the data and store it in a list matrix
    #Convert the list matrix to a np.array to store into DataFrame
    for count in range(shape[0]):
        a1 = []  
        temp = dframe.iloc[count].iloc[1:]    
        
        for i in temp:
            a1.append(i)

        dataMatrix.append(a1)
    
    dataMatrix = np.array(dataMatrix)
    
    #Fix the names for QB, WR, TE, RB
    if(position=='QB' or position=='RB' or position=='TE' or position=='WR' ):
        nameIndex = nameFix(nameIndex, dframe['\xa0'].get("Player")) 
        retframe = DataFrame(dataMatrix, columns = index, index = nameIndex)
    #Fix the names for Kicker and Defense
    elif(position=='DST' or position=='K'):
        dname = dict()
        count = 0
        retframe = DataFrame(dataMatrix, columns = index)
        
        #Create a dictionary of 'fixed' team names, then replace the original with the
        #fixed version
        if(position == 'DST'):
            nameIndex = dframe.get('Team')
            nameIndex = trimName(nameIndex)
        elif(position == 'K'):
            nameIndex = nameFix(nameIndex, dframe.get('Player'))               
                               
        for i in nameIndex:
            dname[count] = i
            count += 1
        #Rename the frame and change the Pts* to Pts
        retframe = (retframe.rename(dname, axis = 'index'))
        retframe = retframe.rename({'Pts*': 'Pts'}, axis='columns')
        
    #Create the dataframe for the keys using keyList argument and concat
    #with the returning frame
    kframe = DataFrame(dkeylist, nameIndex)
    retframe = pd.concat([retframe, kframe], axis=1, join_axes=[retframe.index])    
        
    return(retframe)


In [182]:
def setDataFrame(dframe, keyList, index):
    nameIndex = []  #This is the name of the team or player
    dataMatrix = [] #This is the data going into dataframe
    d1 = dict()
    d2 = dict()
    position = keyList['Pos'] #Get the positionof the data
    retframe = dframe
    
    dkeylist = {
            ('','Pos'): keyList['Pos'], 
            ('', 'PPR'): ('No' if keyList['StdScore'] else 'Yes'),
            ('', 'Week'): keyList['Week'],
            ('', 'Year'): keyList['Year']
         }
    

    
    #For QB, WR, TE, and RB, rename the columns and index to be more legible
    if(position=='QB' or position=='RB' or position=='TE' or position=='WR' ):
        
        d1 = dict(zip(dframe.columns.levels[0], 
            ['Passing', 'Passing','Passing', 'Passing', 'Rushing',\
             'Rushing', 'Rushing', 'Receiving',\
             'Receiving', 'Receiving', 'Receiving',\
             'Fumbles', 'Fumbles', 'Passing',\
             'Passing', 'Rushing', '']))
        
        nameIndex = nameFix(nameIndex, dframe['\xa0'].get("Player")) 
        #retframe = dframe.rename(columns=d1, level=0)
        
        d2 = dict(zip(retframe[('\xa0','Player')], nameIndex))
        retframe = retframe.set_index(('\xa0','Player'))

    #Fix the names for Kicker and Defense
    elif(position=='DST' or position=='K'):
    
        #Create a dictionary of 'fixed' team names, then replace the original with the
        #fixed version
        if(position == 'DST'):
            nameIndex = dframe.get('Team')
            nameIndex = trimName(nameIndex)
            
            d2 = dict(zip(retframe[('Team')], nameIndex))
            retframe = retframe.set_index(('Team'))
            
        elif(position == 'K'):
            nameIndex = nameFix(nameIndex, dframe.get('Player'))   
            d2 = dict(zip(retframe[('Player')], nameIndex))
            retframe = retframe.set_index(('Player'))
    
    #Take away the axis name. 
    #Rename the index using the fixed player/team name
    #Change the column names and remove the * in the Pts column
    retframe = retframe.rename_axis('')
    retframe = retframe.rename(d2, axis='index')
    retframe.columns = index    
    retframe = retframe.rename({'Pts*': 'Pts'}, axis='columns')
    #Create the dataframe for the keys using keyList argument and concat
    #with the returning frame
    kframe = DataFrame(dkeylist, nameIndex)
    retframe = pd.concat([retframe, kframe], axis=1, join_axes=[retframe.index])    
        
    return(retframe)


In [224]:
def writePickle(data, filename):
    file = open(filename, 'wb') #w = write, b= binary mode
    pickle.dump(data, file)
    file.close()
    

In [232]:
def writeCSV(data, filename):
    typeTester = DataFrame()
    
    if(type(data) == type(typeTester)):
        data.to_csv(filename)
    
    

In [15]:
#MAIN STARTS HERE

In [184]:
columns = pd.MultiIndex.from_tuples(masterTup)
frameMaster = DataFrame(columns = columns)
frameArray = []

try:
    tframeArray
except NameError:   
    tframeArray = pd.read_pickle('testpickle')

In [185]:


test_count = 0
if(len(frameMaster.index) == 0):
    for tup in tframeArray:
        index = createIndex(tup[KEYS])
        dframe = setDataFrame(tup[DATA], tup[KEYS], index)
        frameArray.append(dframe)
        frameMaster = pd.concat([frameMaster, dframe])
    
        test_count += 0
        if test_count == 5:
            break
        

# Test section

In [298]:
#RB = frameMaster.loc[frameMaster.index=='Ezekiel Elliott']
#rushing_yards = frameMaster['Rushing','Yds'] 
#year = Zeek['','Year'] == 2017


#noppr = Zeek['','PPR'] == 'No'

#Zeek[year & noppr]['Fumbles','FL'].sum()


noppr = frameMaster['','PPR'] == 'No'
pos = frameMaster['','Pos'] == 'QB'
year = frameMaster['', 'Year'] == 2017
yds = frameMaster['Passing','Yds'] > 300
tds = frameMaster['Passing', 'TD'] <=1
player = (frameMaster.index == 'Tom Brady') | (frameMaster.index == 'Russell Wilson')
#player2 = frameMaster.index == 'Russell Wilson'

#player3 = player | player2

start_time = time.time()

fframe = frameMaster[pos & noppr & yds & year & player & tds]
elapsed_time = time.time() - start_time
print(elapsed_time)

0.003080129623413086


In [301]:
writeCSV(frameMaster, "ffbDFrame.csv")